In [177]:
using JLD
using SparseArrays
using LinearAlgebra
using PyCall
using TensorOperations
using FFTW
using DifferentialEquations
using SpecialFunctions
using MKL
using QuadGK
using Plots 
gspec=pyimport("matplotlib.gridspec")
patches=pyimport("matplotlib.patches")
mpimg=pyimport("matplotlib.image")

include("./src/twm_simulate.jl")

BLAS.set_num_threads(5)
println(BLAS.get_config())
println(Threads.nthreads())

function gaussian_SH_pulse(zlist,σ)
    vect = [exp(-(z/σ)^2/2) for z in zlist]
    vect ./= norm(vect)
    return vect
end

LBTConfig()
10


gaussian_SH_pulse (generic function with 1 method)

In [184]:
L = 8.0   # Size of the system. Open boundary condition.
n_half = 100   # Number of spatial bins per band. Total size of the MPS will be 2 times this number, because we have both signal and pump modes.
n_l = n_half*2   # Total number of spatial bins.
Δz = L/n_half   # Size of spatial bins
klist = [2*pi*l/L for l in -floor(Int64,n_half/2):1:ceil(Int64,n_half/2)-1]   # Wavespace coordinate
zlist = collect(range(-L/2,stop=L/2-Δz,length=n_half));   # Spatial coordinate
    
mmax = 4   # Maximum Fock space cutoff. 4 is usually enough.
n_fock = 4   # Fock space cutoff
χ = 15   # MPS bond dimension. Depends on the simulation.

β = 2   # GVD for the pump mode, relative to the signal GVD.
J = 1/(2*Δz^2)/(4pi^2)   # Nearnest-neighbor interactions between spatial bins. The value is proportionla to signal GVD.
V = 1/sqrt(Δz)   # Nonlinear coupling between local signal and pump bins.
Δ = 1.9   # Phase-mismatch between signal and pump




1.9

In [185]:
using Trapz
efficiency = []
n_b = 0.02
tmax = 2.5
nt = 250 # Steps
saveat = [i for i in 1:2:nt]
th = 1e-10 # Truncation for SVD
α_a = 0.0
σ = 0.2

flist = gaussian_SH_pulse(zlist,σ)   # Pump field distribution

F0 = maximum(flist)

flist_a = [i%2==1 ? flist[round(Int64,(i+1)/2)] : 0.0im for i in 1:n_l]
flist_a ./= norm(flist_a)
flist_b = [i%2==0 ? flist[round(Int64,(i)/2)] : 0.0im for i in 1:n_l]
flist_b ./= norm(flist_b)
α_b = sqrt(n_b)

Γlist,λlist = fock_twm(0,1,flist_a,flist_b,n_l,χ,mmax);

Γset,λset,norm_list = simulate_twm!(Γlist,λlist,J,β,V,Δ,tmax,nt,saveat;threshold=th);
αset_a = [[mean(Γset[si],λset[si],ci) for ci in 1:2:n_l] for si in eachindex(saveat)]
αset_b = [[mean(Γset[si],λset[si],ci) for ci in 2:2:n_l] for si in eachindex(saveat)]
g1set_a = [[g1(Γset[si],λset[si],ci) for ci in 1:2:n_l] for si in eachindex(saveat)]
g1set_b = [[g1(Γset[si],λset[si],ci) for ci in 2:2:n_l] for si in eachindex(saveat)];

push!(efficiency, [trapz(zlist, real(g1set_a[i])) for i in 1:length(saveat)]/trapz(zlist, real(g1set_b[1]))/2)


1
  0.796650 seconds (379.83 k allocations: 436.498 MiB, 20.01% gc time)
  1.011101 seconds (363.96 k allocations: 436.100 MiB)
2
  0.844311 seconds (362.33 k allocations: 436.053 MiB)
  1.754027 seconds (372.11 k allocations: 436.307 MiB, 54.50% gc time)
3
  1.082073 seconds (379.33 k allocations: 436.486 MiB)
  1.666984 seconds (386.77 k allocations: 436.680 MiB, 53.03% gc time)
4
  1.407259 seconds (388.20 k allocations: 436.711 MiB, 38.94% gc time)
  1.988846 seconds (388.76 k allocations: 436.731 MiB, 56.36% gc time)
5
  0.772807 seconds (393.64 k allocations: 436.849 MiB)
  1.769367 seconds (400.89 k allocations: 437.039 MiB, 53.92% gc time)
6
  1.370631 seconds (401.05 k allocations: 437.038 MiB, 40.64% gc time)
  0.882024 seconds (403.25 k allocations: 437.099 MiB)
7
  1.720095 seconds (404.68 k allocations: 437.130 MiB, 50.60% gc time)
  0.970017 seconds (407.95 k allocations: 437.218 MiB)
8
  0.765815 seconds (411.92 k allocations: 437.314 MiB)


Excessive output truncated after 524291 bytes.

In [ ]:
t = range(0,tmax, length(efficiency[1]))
p = plot(xlabel = "gt", ylabel = "Conversion efficiency")
for i in 1:1
    if i == 1
        plot!(t, efficiency[i] .^2, label = "n̄ = Single-Photon Pump")
    else
        plot!(t, efficiency[i] .^2, label = "n̄ = $(n_bs[i])")
    end
end
display(p)
#savefig("Figures/Use")